In [37]:
##checkpoint_eoc csv file
##merge on book, chapter, student_id 
##filter per chapter, 
##eoc number of corrects, it's n_correct/ n_possible 
##filter on the response type and disregard the  na's


#merge checkpoint_eoc.csv and checkpoint_pulse.cdv on stunt_id, book, chapter
#get the EOC, n_attempy from checkpoint_eoc.csv


import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

d1 = pd.read_csv("C:\\Users\\HP\\Desktop\\datafest\\Random Sample of Data Files_03_04\\checkpoints_eoc.csv")
d2 = pd.read_csv("C:\\Users\\HP\\Desktop\\datafest\\Random Sample of Data Files_03_04\\checkpoints_pulse.csv")


# Merge the two dataframes on student_id, book, chapter, including EOC and n_attempt
d3 = pd.merge(d1, d2, on=['student_id', 'book', 'chapter_number'])

# Sort per book and chapter
d3 = d3.sort_values(['book', 'chapter_number'])

#drop 

# Print 10 samples
#d3.head(10)

#drop class_id, n_possible, n_correct, response, construct, release 
d3 = d3.drop(['class_id_x', 'n_possible', 'n_correct', 'release','class_id_y','institution_id','response','n_attempt','response','construct'], axis=1)

#just take the consttruct= Cost
#d3 = d3[d3['construct'] == 'Cost']

#drop nan values
#d3 = d3.dropna(subset=['response'])
print(d3.head(10))

#create a new csv file at the same path






                               student_id  chapter_number  \
44   041a771b-41cc-4b36-928d-ec321f918db5               2   
45   041a771b-41cc-4b36-928d-ec321f918db5               2   
46   041a771b-41cc-4b36-928d-ec321f918db5               2   
47   041a771b-41cc-4b36-928d-ec321f918db5               2   
84   04347fe5-e55a-4adc-86f2-a156ddfcfc13               2   
85   04347fe5-e55a-4adc-86f2-a156ddfcfc13               2   
86   04347fe5-e55a-4adc-86f2-a156ddfcfc13               2   
87   04347fe5-e55a-4adc-86f2-a156ddfcfc13               2   
128  045d0e87-585c-4126-a2c0-c744effb11b2               2   
129  045d0e87-585c-4126-a2c0-c744effb11b2               2   

                                                  book       EOC  
44   College / Advanced Statistics and Data Science...  0.673469  
45   College / Advanced Statistics and Data Science...  0.673469  
46   College / Advanced Statistics and Data Science...  0.673469  
47   College / Advanced Statistics and Data Science...  0.67

In [44]:
import pandas as pd

# Assuming d3 is your DataFrame after any necessary merging and cleaning steps.

# Calculate the mean EOC for each book and chapter
mean_eoc_per_chapter = d3.groupby(['book', 'chapter_number'])['EOC'].mean().reset_index()

# Define the difficulty metric as the inverse of the mean EOC score
mean_eoc_per_chapter['difficulty'] = 1 - mean_eoc_per_chapter['EOC']

# Merge this difficulty metric back into the original DataFrame on 'book' and 'chapter_number'
d3_with_difficulty = d3.merge(mean_eoc_per_chapter[['book', 'chapter_number', 'difficulty']], 
                              on=['book', 'chapter_number'], 
                              how='left')

# Create a new DataFrame with just the book, chapter number, and difficulty
difficulty_only_df = mean_eoc_per_chapter[['book', 'chapter_number', 'difficulty']]

# Now you have d3_with_difficulty which includes the original data plus a difficulty column,
# and difficulty_only_df which contains only the book, chapter, and difficulty.
print(difficulty_only_df.head(10))

# Save the difficulty_only_df to a new CSV file on the samepath with this "C:\\Users\\HP\\Desktop\\datafest\\Random Sample of Data Files_03_04\\checkpoints_pulse.csv"
difficulty_only_df.to_csv("C:\\Users\\HP\\Desktop\\datafest\\Random Sample of Data Files_03_04\\difficulty_only.csv", index=False)

#also  save the d3_with_difficulty to a new CSV file on the samepath with this "C:\\Users\\HP\\Desktop\\datafest\\Random Sample of Data Files_03_04\\checkpoints_pulse.csv"
d3_with_difficulty.to_csv("C:\\Users\\HP\\Desktop\\datafest\\Random Sample of Data Files_03_04\\d3_with_difficulty.csv", index=False)




                                                book  chapter_number  \
0  College / Advanced Statistics and Data Science...               2   
1  College / Advanced Statistics and Data Science...               3   
2  College / Advanced Statistics and Data Science...               4   
3  College / Advanced Statistics and Data Science...               5   
4  College / Advanced Statistics and Data Science...               6   
5  College / Advanced Statistics and Data Science...               7   
6  College / Advanced Statistics and Data Science...               8   
7  College / Advanced Statistics and Data Science...               9   
8  College / Advanced Statistics and Data Science...              10   
9  College / Advanced Statistics and Data Science...              11   

   difficulty  
0    0.273057  
1    0.306468  
2    0.347523  
3    0.334382  
4    0.379361  
5    0.370850  
6    0.369938  
7    0.405785  
8    0.414719  
9    0.492801  


In [ ]:
'''
# Since we're working with a simulated dataset, we'll create conditions to remove outliers based on your description.
# In your actual data, you would apply these conditions to your d3 DataFrame.

# Let's first simulate the dataset with 'EOC', 'response', and 'n_attempt'.
np.random.seed(0)  # Seed for reproducibility
dummy_data = {
    'EOC': np.random.uniform(0, 1, 1000),  # 1000 random EOC scores between 0 and 1
    'response': np.random.choice([0, 1, 2, 3, 4, 5], 1000),  # 1000 random responses between 0 and 5
    'n_attempt': np.random.randint(1, 300, 1000)  # 1000 random attempts between 1 and 300
}

# This is the simulated DataFrame
d3 = pd.DataFrame(dummy_data)

# Condition to remove outliers in 'response' and 'EOC' where response is high and EOC is low or vice versa.
high_response_low_eoc = (d3['response'] > 4) & (d3['EOC'] < 0.2)
low_response_high_eoc = (d3['response'] < 1) & (d3['EOC'] > 0.8)
outliers_condition = ~(high_response_low_eoc | low_response_high_eoc)

# Apply the condition to filter out the outliers from 'response' and 'EOC'
d3_filtered = d3[outliers_condition]

# Additional condition to disregard more than 250 attempts.
d3_filtered = d3_filtered[d3_filtered['n_attempt'] <= 250]

# Calculate the correlation again on the filtered data
correlation_filtered = d3_filtered['EOC'].corr(d3_filtered['n_attempt'])
correlation_2_filtered = d3_filtered['response'].corr(d3_filtered['EOC'])
print("Correlation between EOC and number of attempts (filtered):", correlation_filtered)
print("Correlation between response and EOC (filtered):", correlation_2_filtered)

# Let's visualize the filtered data
plt.figure(figsize=(10, 6))
plt.scatter(d3_filtered['response'], d3_filtered['EOC'], alpha=0.5)
plt.title('Scatter Plot of EOC vs. Response (Outliers Removed)')
plt.xlabel('Response (Perceived Investment of Time)')
plt.ylabel('EOC (Success Rate)')
plt.grid(True)
plt.show()
'''
